In [5]:
import torch
import torchvision
torch.cuda.is_available()

True

In [2]:
#importing reqd libraries
import numpy as np
# import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# from google.colab import drive
from torchvision import transforms, datasets
from torch.utils.data import DataLoader,random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os
# import torchvision

# Mount your Google Drive
# drive.mount('/content/drive')

# Set the path to your data directory
# data_dir = "/home/patel.avni/Plastics_Classification"

# Define the batch size and image size
# batch_size = 16
# img_size = (224, 224)


In [4]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
import random
import shutil
from torch.utils.data import  SubsetRandomSampler
# Set the path to your data directory
data_dir = "E:/Computer_Vision/Final_project_Computer_vision/Plastics_Classification_3"

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.7552, 0.7162, 0.6846),(0.2468, 0.2586, 0.2788))
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.7532, 0.7138, 0.6792),(0.2489, 0.2587, 0.2797))
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.7591, 0.7196, 0.6895),(0.2456, 0.2572, 0.2756))
])


# Compute the mean and standard deviation of the train dataset
# Load the train and test sets
 # create two dataset objects, train_dataset and val_dataset, from the image data located in the train directory within the data_dir.

train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

classes = ["heavy_plastic", "no_image", "no_plastic", "some_plastic"]


train_dataset = ImageFolder(train_dir, transform=train_transforms)

train_class_counts = [0] * len(train_dataset.classes)
for _, label in train_dataset:
    train_class_counts[label] += 1

train_class_weights = [1.0 / count for count in train_class_counts]
train_sample_weights = [train_class_weights[label] for _, label in train_dataset]

train_sampler = SubsetRandomSampler(np.arange(len(train_dataset)))
train_loader = DataLoader(train_dataset, batch_size=8, sampler=train_sampler)

# Create the validation dataset and data loader
val_dataset = ImageFolder(val_dir, transform=val_transforms)

val_class_counts = [0] * len(val_dataset.classes)
for _, label in val_dataset:
    val_class_counts[label] += 1

val_class_weights = [1.0 / count for count in val_class_counts]
val_sample_weights = [val_class_weights[label] for _, label in val_dataset]

val_sampler = SubsetRandomSampler(np.arange(len(val_dataset)))
val_loader = DataLoader(val_dataset, batch_size=8, sampler=val_sampler)


# Create the test dataset and data loader
test_dataset = ImageFolder(test_dir, transform=test_transforms)

test_class_counts = [0] * len(test_dataset.classes)
for _, label in test_dataset:
    test_class_counts[label] += 1

test_class_weights = [1.0 / count for count in test_class_counts]
test_sample_weights = [test_class_weights[label] for _, label in test_dataset]

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


FileNotFoundError: ignored

In [ ]:
# Compute the class weights for loss function
class_count = [0] * len(classes)
for _, labels in train_dataset:
    for i in range(len(classes)):
        if labels == i:
            class_count[i] += 1
total_count = sum(class_count)
class_weights = [total_count / (len(classes) * count) for count in class_count]
print("Class weights:", class_weights)

In [ ]:


def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    best_val_acc=0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_total = 0
        train_correct = 0
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += targets.size(0)
            train_correct += (predicted == targets).sum().item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()

        # Update the learning rate scheduler
        scheduler.step()


        # Print the training and validation loss and accuracy
        train_loss /= len(train_loader.dataset)
        train_acc = 100.0 * train_correct / train_total
        val_loss /= len(val_loader.dataset)
        val_acc = 100.0 * val_correct / val_total
        print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

        if val_acc > best_val_acc:
            torch.save(model.state_dict(),'best_model.pth')
            best_val_acc=val_acc


In [ ]:
def evaluate_model(model, dataloader):
    """
    Evaluate the performance of the model on a given dataset.

    Args:
        model (torch.nn.Module): The PyTorch model to evaluate.
        dataloader (torch.utils.data.DataLoader): The PyTorch DataLoader object to use for iterating over the dataset.

    Returns:
        float: The accuracy of the model on the given dataset.
    """
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = torch.tensor(class_weights, device=device) # move to CUDA

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import random
import torchvision.models as models
import torch.nn as nn
import optuna
import torch.nn.init as init
from efficientnet_pytorch import EfficientNet


# Define the hyperparameters to search over
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-2)
#     dropout = trial.suggest_uniform("dropout", 0.0, 0.5)
    # epochs = trial.suggest_int("epochs", 5, 20)
    # step_size = trial.suggest_int("step_size", 1, 10)
    # gamma = trial.suggest_loguniform("gamma", 1e-5, 1e-2)
#     batch_size = trial.suggest_categorical("batch_size", [ 4,8])

    # Create data loaders with the specified batch size
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Create a new instance of the EfficientNet-B0 model
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

# Modify the model
in_features = efficientnet._fc.in_features
efficientnet._fc = nn.Sequential(
    nn.Dropout(0.1693343055004149),
    nn.Linear(in_features, 4)
)

# Move the model and data to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet = efficientnet.to(device)

# Define the loss function, optimizer and learning rate scheduler
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights))

#     if optimizer_name == "Adam":
#         optimizer = optim.Adam(efficientnet.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = optim.Adamax(efficientnet.parameters(), lr= 0.00045757341361, weight_decay=0.0016725928825570783)
#     elif optimizer_name == "Adagrad":
#         optimizer = optim.Adagrad(efficientnet.parameters(), lr=learning_rate, weight_decay=weight_decay)

scheduler = StepLR(optimizer, step_size=8, gamma=0.1)

# Train the model
train_model(efficientnet, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25)

# Evaluate the model on the validation set
val_acc = evaluate_model(efficientnet, val_loader)




# # study = optuna.create_study(direction="maximize")
# study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=20)

# # Print the best validation accuracy and best hyperparameters
# best_trial = study.best_trial
# print(f'Best validation accuracy: {best_trial.value:.4f}')
# print("Best hyperparameters:")
# for key, value in best_trial.params.items():
#     print(f"{key}: {value}")


In [ ]:
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
from torchvision.datasets import ImageFolder
import os
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




data_dir = "E:/Computer_Vision/Final_project_Computer_vision/Plastics_Classification_3"

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.7591, 0.7196, 0.6895),(0.2456, 0.2572, 0.2756))
])



# Define a function to compute the accuracy of the model on a given dataset
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient tracking during inference
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Load the test dataset
# test_dataset = MyDataset(test_data, test_labels)
test_dataset= ImageFolder(os.path.join(data_dir, "test"), transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Load the best model found by Optuna
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
in_features = efficientnet._fc.in_features
efficientnet._fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_features, 4)
)
efficientnet.load_state_dict(torch.load("best_model.pth"))
efficientnet = efficientnet.to(device)

# Evaluate the model on the test set
test_acc = evaluate_model(efficientnet, test_loader)
print(f'Test accuracy: {test_acc:.4f}')


In [ ]:
print(f"Number of batches in test loader: {len(test_loader)}")



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# Set the model to evaluation mode
efficientnet.eval()

test_iterator = iter(test_loader)
test_data = [next(test_iterator) for i in range(150)]

for i, (images, labels) in enumerate(test_data):
    images = images.to(device)

    with torch.no_grad():
        outputs = efficientnet(images)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)

    images = images.cpu().numpy()
    labels = labels.cpu().numpy()
    preds = preds.cpu().numpy()
    probs = probs.cpu().numpy()

    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    for j, image in enumerate(images):
        pred_class = class_names[preds[j]]
        true_class = class_names[labels[j]]
        confidence = probs[j, preds[j]]
        axs[j // 4, j % 4].imshow(np.transpose(image, (1, 2, 0)))
        axs[j // 4, j % 4].set_title(f"Predicted: {pred_class}\nTrue: {true_class}\nConfidence: {confidence:.3f}")
        axs[j // 4, j % 4].axis("off")

    plt.show()



In [ ]:
import torch.nn as nn
from captum.attr import GuidedBackprop

# Define a GuidedBackprop attribution algorithm
guided_backprop = GuidedBackprop(efficientnet)

# Choose an image index for which to generate a saliency map
image_index = 0

# Get the corresponding image and label from the test data
image, label = test_data[image_index]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image = image.to(device)
label = label.to(device)

# Extract the first image from the batch and add a batch dimension
image = image[0].unsqueeze(0)

# Get the predicted class probabilities for the given image
with torch.no_grad():
    output = efficientnet(image)
    probs = nn.functional.softmax(output, dim=1)

# Get the index of the class with the highest probability
target_class = torch.argmax(probs, dim=1)

# Calculate the saliency map for the chosen image
saliency_map = guided_backprop.attribute(image, target=target_class)
saliency_map = saliency_map.to(device)

# Convert the saliency map to a grayscale image and show it
# print("Input image shape:", image.shape)  # Debugging line
saliency_map = np.transpose(saliency_map.squeeze().cpu().detach().numpy(), (1, 2, 0))
saliency_map = np.maximum(saliency_map, 0)
saliency_map /= saliency_map.max()
plt.imshow(saliency_map, cmap="gray")
plt.show()


In [ ]:
import torch.nn as nn
from captum.attr import GuidedBackprop

# Define a GuidedBackprop attribution algorithm
guided_backprop = GuidedBackprop(efficientnet)

# Generate saliency maps for all images in the test loader
for image, label in test_loader:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = image.to(device)
    label = label.to(device)

    # Extract the first image from the batch and add a batch dimension
    image = image[0].unsqueeze(0)

    # Get the predicted class probabilities for the given image
    with torch.no_grad():
        output = efficientnet(image)
        probs = nn.functional.softmax(output, dim=1)

    # Get the index of the class with the highest probability
    target_class = torch.argmax(probs, dim=1)

    # Calculate the saliency map for the chosen image
    saliency_map = guided_backprop.attribute(image, target=target_class)
    saliency_map = saliency_map.to(device)

    # Convert the saliency map to a grayscale image and show it
    saliency_map = np.transpose(saliency_map.squeeze().cpu().detach().numpy(), (1, 2, 0))
    saliency_map = np.maximum(saliency_map, 0)
    saliency_map /= saliency_map.max()
    plt.imshow(saliency_map, cmap="gray")
    plt.show()


In [ ]:
import torch.nn as nn
from captum.attr import GuidedBackprop

# Define a GuidedBackprop attribution algorithm
guided_backprop = GuidedBackprop(efficientnet)

# Generate saliency maps for all images in the test loader
for image, label in test_loader:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = image.to(device)
    label = label.to(device)

    # Extract the first image from the batch and add a batch dimension
    image = image[0].unsqueeze(0)

    # Get the predicted class probabilities for the given image
    with torch.no_grad():
        output = efficientnet(image)
        probs = nn.functional.softmax(output, dim=1)

    # Get the index of the class with the highest probability
    target_class = torch.argmax(probs, dim=1)

    # Calculate the saliency map for the chosen image
    saliency_map = guided_backprop.attribute(image, target=target_class)
    saliency_map = saliency_map.to(device)

    # Convert the saliency map and original image to numpy arrays
    saliency_map = np.transpose(saliency_map.squeeze().cpu().detach().numpy(), (1, 2, 0))
    saliency_map = np.maximum(saliency_map, 0)
    saliency_map /= saliency_map.max()
    original_image = np.transpose(image.squeeze().cpu().detach().numpy(), (1, 2, 0))

    # Create a figure with two subplots for the original image and saliency map
    fig, axs = plt.subplots(ncols=2, figsize=(10, 5))

    # Display the original image in the left subplot
    axs[0].imshow(original_image)
    axs[0].set_title('Original Image')

    # Display the saliency map in the right subplot
    axs[1].imshow(saliency_map, cmap="gray")
    axs[1].set_title('Saliency Map')

    # Show the figure
    plt.show()


In [ ]:
print("Input image shape:", image.shape)
print("Input image shape:", label.shape)

In [ ]:
def get_embedding(model, dataloader):
    # Set the model to evaluation mode
    model.eval()

    # Create empty lists to store embeddings and labels
    embeddings = []
    labels = []

    # Iterate over the data in the dataloader
    with torch.no_grad():
        for images, target in dataloader:
            images = images.to(device)

            # Get the embeddings from the model
            features = model.extract_features(images)
            embeddings.append(features.cpu().numpy())

            # Get the labels
            labels.append(target.cpu().numpy())

    # Concatenate the embeddings and labels into a single array
    embeddings = np.concatenate(embeddings)
    labels = np.concatenate(labels)

    return embeddings, labels


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Set the model to evaluation mode
efficientnet.eval()

# Get the embeddings of the test set
embeddings = []
labels = []
images = []
with torch.no_grad():
    for batch_images, batch_labels in test_loader:
        batch_images = batch_images.to(device)
        batch_embeddings = efficientnet(batch_images)
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())
        images.append(batch_images.cpu().numpy())
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)
images = np.concatenate(images, axis=0)

# Normalize the images and convert to uint8
images = (images * 255).astype(np.uint8)

# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

# Visualize the embeddings
fig, ax = plt.subplots(figsize=(10, 10))
for i in range(len(class_names)):
    indices = np.where(labels == i)[0]
    x = embeddings_tsne[indices, 0]
    y = embeddings_tsne[indices, 1]
    ax.scatter(x, y, label=class_names[i], alpha=0.5)
plt.title("t-SNE Visualization of Test Set Embeddings")
plt.legend()
plt.show()


In [ ]:
from sklearn.manifold import TSNE
import visdom
import torch
import numpy as np

# Create a Visdom server instance
viz = visdom.Visdom()

# Set the model to evaluation mode
efficientnet.eval()
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}
# Get the embeddings of the test set
embeddings = []
labels = []
images = []
with torch.no_grad():
    for batch_images, batch_labels in test_loader:
        batch_images = batch_images.to(device)
        batch_embeddings = efficientnet(batch_images)
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())
        images.append(batch_images.cpu().numpy())
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)
images = np.concatenate(images, axis=0)

# Normalize the images and convert to uint8
images = (images * 255).astype(np.uint8)

# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)
embeddings_tsne = embeddings_tsne.reshape(-1, 2)


# Add 1 to the labels to ensure they start from 1
labels += 1

# Visualize the embeddings in Visdom
scatter_data = []
for i in range(len(class_names)):
    indices = np.where(labels == i)[0]
    x = embeddings_tsne[indices, 0]
    y = embeddings_tsne[indices, 1]
    label = class_names[i]
    scatter_data.append({'x': x, 'y': y, 'name': label, 'mode': 'markers', 'marker': {'size': 5, 'opacity': 0.5}})
viz.scatter(X=embeddings_tsne, Y=labels, opts=dict(title='t-SNE Visualization of Test Set Embeddings', markersize=5, xlabel='x', ylabel='y',legend=list(class_names.values())))


In [ ]:
import umap
import visdom
import torch
import numpy as np

# Create a Visdom server instance
viz = visdom.Visdom()

# Set the model to evaluation mode
efficientnet.eval()
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}
# Get the embeddings of the test set
embeddings = []
labels = []
images = []
with torch.no_grad():
    for batch_images, batch_labels in test_loader:
        batch_images = batch_images.to(device)
        batch_embeddings = efficientnet(batch_images)
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())
        images.append(batch_images.cpu().numpy())
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)
images = np.concatenate(images, axis=0)

# Normalize the images and convert to uint8
images = (images * 255).astype(np.uint8)

# Perform UMAP dimensionality reduction
umap_reducer = umap.UMAP(n_components=2, random_state=42)
embeddings_umap = umap_reducer.fit_transform(embeddings)

# Add 1 to the labels to ensure they start from 1
labels += 1

# Visualize the embeddings in Visdom
scatter_data = []
for i in range(len(class_names)):
    indices = np.where(labels == i)[0]
    x = embeddings_umap[indices, 0]
    y = embeddings_umap[indices, 1]
    label = class_names[i]
    scatter_data.append({'x': x, 'y': y, 'name': label, 'mode': 'markers', 'marker': {'size': 5, 'opacity': 0.5}})
viz.scatter(X=embeddings_umap, Y=labels, opts=dict(title='UMAP Visualization of Test Set Embeddings', markersize=5, xlabel='x', ylabel='y',legend=list(class_names.values())))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# Set the model to evaluation model
efficientnet.to(device)
efficientnet.eval()

# Define the different sizes of datasets to evaluate, from 10 to the maximum size of the test dataset
dataset_sizes = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1342]

# Create a dictionary to store the evaluation results
evaluation_results = {}

for size in dataset_sizes:
    # Select a random subset of the test dataset
    test_subset = torch.utils.data.random_split(test_dataset, [size, len(test_dataset) - size])[0]
    test_loader = torch.utils.data.DataLoader(test_subset, batch_size=8, shuffle=False)

    test_iterator = iter(test_loader)
    test_data = [next(test_iterator) for i in range(len(test_loader))]

    correct_predictions = 0
    total_predictions = 0

    for i, (images, labels) in enumerate(test_data):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = efficientnet(images)
            _, preds = torch.max(outputs, 1)

        correct_predictions += torch.sum(preds == labels.data)
        total_predictions += len(labels)

    accuracy = correct_predictions.double() / total_predictions
    evaluation_results[size] = accuracy.item()

# Plot the evaluation results
plt.plot(list(evaluation_results.keys()), list(evaluation_results.values()))
plt.xlabel('Dataset size')
plt.ylabel('Accuracy')
plt.show()


In [ ]:
import torch.nn.functional as F

# Define epsilon for FGSM
epsilon = 0.8

# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

test_iterator = iter(test_loader)
test_data = [next(test_iterator) for i in range(7)]

for i, (images, labels) in enumerate(test_data):
    images = images.to(device)
    labels = labels.to(device)

    # Evaluate the model's accuracy on original examples
    with torch.no_grad():
        outputs = efficientnet(images)
        _, preds = torch.max(outputs, 1)
    accuracy_original = torch.sum(preds == labels).item() / len(labels) *100

    # Generate adversarial examples using FGSM
    images.requires_grad = True
    outputs = efficientnet(images)
    loss = F.cross_entropy(outputs, labels)
    loss.backward()
    images_grad = torch.sign(images.grad)
    images_adversarial = images + epsilon * images_grad
    images_adversarial = torch.clamp(images_adversarial, 0, 1)

    # Evaluate the model's accuracy on adversarial examples
    with torch.no_grad():
        outputs = efficientnet(images_adversarial)
        _, preds_adv = torch.max(outputs, 1)
    accuracy_adversarial = torch.sum(preds_adv == labels).item() / len(labels) *100

    images = images.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    preds = preds.detach().cpu().numpy()
    preds_adv = preds_adv.detach().cpu().numpy()
    images_adversarial = images_adversarial.detach().cpu().numpy()

    fig, axs = plt.subplots(nrows=2, ncols=len(images), figsize=(15, 5))
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    for j, image in enumerate(images):
        pred_class = class_names[preds[j]]
        true_class = class_names[labels[j]]
        axs[0, j].imshow(np.transpose(image, (1, 2, 0)))
        axs[0, j].set_title(f"Original\nPredicted: {pred_class}\nTrue: {true_class}",fontsize=8)
        axs[0, j].axis("off")

    for j, image in enumerate(images_adversarial):
        pred_class_adv = class_names[preds_adv[j]]
        true_class_adv = class_names[labels[j]]
        axs[1, j].imshow(np.transpose(image, (1, 2, 0)))
        axs[1, j].set_title(f"Adversarial\nPredicted: {pred_class_adv}\nTrue: {true_class_adv}",fontsize=8)
        axs[1, j].axis("off")

    print(f"Accuracy on original examples: {accuracy_original:.4f}%")
    print(f"Accuracy on adversarial examples: {accuracy_adversarial:.4f}%")
    plt.show()




In [ ]:
true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = efficientnet(images)
        _, preds = torch.max(outputs, 1)
        true_labels += labels.cpu().numpy().tolist()
        predicted_labels += preds.cpu().numpy().tolist()

cm = confusion_matrix(true_labels, predicted_labels)
sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu", xticklabels=class_names.values(), yticklabels=class_names.values())
plt.title("Confusion Matrix for Test Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()


In [ ]:
# import torch
# import matplotlib.pyplot as plt
# import numpy as np
# import cv2

# # Define a dictionary that maps label indices to class names
# class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# # Set the model to evaluation mode
# efficientnet.eval()

# # Define the loss function as the cross-entropy loss
# loss_fn = torch.nn.CrossEntropyLoss()

# # Define a function to compute the saliency map
# def compute_saliency_map(inputs, target_class):
#     inputs.requires_grad_()
#     outputs = efficientnet(inputs)
#     loss = loss_fn(outputs, torch.tensor(target_class).to(device))
#     loss.backward()
#     saliency_map = torch.abs(inputs.grad).max(dim=1)[0]
#     saliency_map = torch.stack([saliency_map]*3, dim=1)
#     return saliency_map

# test_iterator = iter(test_loader)
# test_data = [next(test_iterator) for i in range(7)]

# for i, (images, labels) in enumerate(test_data):
#     images = images.to(device)

#     with torch.no_grad():
#         outputs = efficientnet(images)
#         _, preds = torch.max(outputs, 1)

#     images = images.cpu().numpy()
#     labels = labels.cpu().numpy()
#     preds = preds.cpu().numpy()

#     fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(10, 7))
#     fig.subplots_adjust(hspace=0.5, wspace=0.3)
#     for j, image in enumerate(images):
#         if j >= axs.shape[1]:
#             break
#         pred_class = class_names[preds[j]]
#         true_class = class_names[labels[j]]
#         axs[0, j].imshow(np.transpose(image, (1, 2, 0)))
#         axs[0, j].set_title(f"Predicted: {pred_class}\nTrue: {true_class}")
#         axs[0, j].axis("off")

#         # Compute the saliency map
#         saliency_map = compute_saliency_map(torch.from_numpy(images).clone().to(device).requires_grad_(), torch.from_numpy(labels).clone().to(device))

#         axs[1, j].imshow(np.transpose(saliency_map[j].cpu().numpy(), (1, 2, 0)))

#         axs[1, j].set_title("Saliency Map")
#         axs[1, j].axis("off")

#         heatmap = np.uint8(255 * np.transpose(saliency_map.cpu().numpy()[j], (1, 2, 0)))
#         heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
# #         plt.imshow(heatmap, cmap='jet')
# #         plt.axis('off')
#         heatmap = np.float32(heatmap) / 255
#         heatmap = heatmap[..., ::-1]  # Convert BGR to RGB
#         superimposed_img = np.float32(np.transpose(image, (1, 2, 0))) / 255 + 0.5 * heatmap
#         superimposed_img /= superimposed_img.max()
#         axs[2, j].imshow(np.transpose(superimposed_img, (0, 1, 2)))
#         axs[2, j].set_title("Superimposed Image")
#         axs[2, j].axis("off")

#     plt.show()


In [ ]:
print(images.shape) # prints the shape of the image array
print(saliency_map.shape) # prints the shape of the saliency map


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.utils as vutils
from efficientnet_pytorch import EfficientNet

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the EfficientNet model
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
efficientnet.to(device)

# Visualize the filters of the first convolutional layer
conv1_filters = efficientnet._conv_stem.weight.data.cpu()
grid = vutils.make_grid(conv1_filters, nrow=8, normalize=True, scale_each=True)

# Show the grid of filters
plt.figure(figsize=(10, 10))
plt.imshow(np.transpose(grid, (1, 2, 0)))
plt.axis("off")
plt.title("Filters of the first convolutional layer")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.utils as vutils
from efficientnet_pytorch import EfficientNet

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the EfficientNet model
# efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
# efficientnet.to(device)

# Define the convolutional layers to visualize
conv_layers = [efficientnet._conv_stem, efficientnet._blocks[0]._depthwise_conv,
               efficientnet._blocks[1]._depthwise_conv]

# Visualize the filters of each convolutional layer
fig = plt.figure(figsize=(12, 8))
for i, layer in enumerate(conv_layers):
    filters = layer.weight.data.cpu()
    grid = vutils.make_grid(filters, nrow=8, normalize=True, scale_each=True)
    ax = fig.add_subplot(1, len(conv_layers), i+1)
    ax.imshow(np.transpose(grid, (1, 2, 0)))
    ax.axis("off")
    ax.set_title(f"Filters of Conv{i+1}")

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Set the model to evaluation mode
efficientnet.eval()

# Initialize lists to store the true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate over the test data
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = efficientnet(images)
        _, preds = torch.max(outputs, 1)

    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(preds.cpu().numpy())

# Convert the true labels and predicted labels to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate the classification report, which includes precision, recall, F1-score, and support
report = classification_report(true_labels, predicted_labels, target_names=class_names.values())

print(report)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_labels, predicted_labels)
print(cm)
# Create a heatmap from the confusion matrix
sns.heatmap(cm, annot=True, cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import shap
import torchvision.transforms.functional as TF

# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# Set the model to evaluation mode
efficientnet.eval()

# Load the test data into a list
test_images = []
test_iterator = iter(test_loader)
for i in range(len(test_loader)):
    images, _ = next(test_iterator)
    test_images.append(images)

# Convert the list to a numpy array
test_images = np.concatenate(test_images, axis=0)

# Remove the first dimension
test_images = test_images[0]

# Convert the numpy array to a PyTorch tensor
to_tensor = transforms.ToTensor()
test_images = to_tensor(test_images)
# test_images = test_images.numpy()

# Load the SHAP explainer
explainer = shap.KernelExplainer(efficientnet, test_images.numpy())

test_images = test_images.numpy()
test_iterator = iter(test_loader)
test_data = [next(test_iterator) for i in range(7)]

for i, (images, labels) in enumerate(test_data):
    images = images.to(device)

    with torch.no_grad():
        outputs = efficientnet(images)
        _, preds = torch.max(outputs, 1)

    images = images.cpu().numpy()
    labels = labels.cpu().numpy()
    preds = preds.cpu().numpy()

    # Compute the SHAP values for the first 5 images in the batch
    shap_values = explainer.shap_values(images[:5])

    # Plot the original images and their corresponding SHAP values side by side
    fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(8, 16))
    for j in range(5):
        axs[j, 0].imshow(np.transpose(images[j], (1, 2, 0)))
        axs[j, 0].axis('off')
        axs[j, 0].set_title(f"Original image for predicted class: {class_names[preds[j]]}")

        shap.image_plot(shap_values[j], np.transpose(images[j], (1, 2, 0)), show=False, ax=axs[j, 1])
        axs[j, 1].axis('off')
        axs[j, 1].set_title(f"SHAP values for predicted class: {class_names[preds[j]]}")

    plt.tight_layout()
    plt.show()


In [ ]:
print(test_images.shape)


In [ ]:
# from cam import cam

# # Define a dictionary that maps label indices to class names
# class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# test_iterator = iter(test_loader)
# test_data = [next(test_iterator) for i in range(7)]

# for i, (images, labels) in enumerate(test_data):
#     images = images.to(device)

#     fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))
#     fig.subplots_adjust(hspace=0.5, wspace=0.3)
#     for j, image in enumerate(images):
#         # Generate the CAM heatmap
#         heatmap = cam(image.unsqueeze(0), efficientnet)

#         # Apply the heatmap to the original image
#         image = image.cpu().numpy().transpose(1, 2, 0)
#         image = cv2.resize(image, (224, 224))
#         heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
#         heatmap = np.float32(heatmap) / 255
#         image_with_heatmap = heatmap + np.float32(image)
#         image_with_heatmap = image_with_heatmap / np.max(image_with_heatmap)

#         # Get the predicted and true classes
#         with torch.no_grad():
#             outputs = efficientnet(torch.unsqueeze(image, 0))
#             _, preds = torch.max(outputs, 1)
#         pred_class = class_names[preds[0].item()]
#         true_class = class_names[labels[j]]

#         # Plot the image with heatmap and label
#         axs[j // 4, j % 4].imshow(image_with_heatmap)
#         axs[j // 4, j % 4].set_title(f"Predicted: {pred_class}\nTrue: {true_class}")
#         axs[j // 4, j % 4].axis("off")

#     plt.show()


In [ ]:
def gradcam(image, model, layer_name):
    """
    Generates a GradCAM saliency map for the input image.

    Args:
        image (torch.Tensor): input image tensor.
        model (torch.nn.Module): trained PyTorch model.
        layer_name (str): name of the layer to generate the heatmap from.

    Returns:
        heatmap (np.ndarray): GradCAM heatmap.
    """
    # Set the model to evaluation mode
    model.eval()

    # Get the image size
    _, _, h, w = image.shape

    # Forward pass
    output = model(image)

    # Get the index of the predicted class
    _, pred_idx = torch.max(output, 1)

    # Get the weights of the output layer
    weights = list(model.parameters())[-2]

    # Get the feature maps from the specified layer
    activations = {}
    def forward_hook(module, input, output):
        activations[layer_name] = output.detach()
    handle = model._modules[layer_name].register_forward_hook(forward_hook)
    feature_maps = model(image)
    handle.remove()

    # Calculate the gradients
    one_hot = torch.zeros_like(output)
    one_hot[0][pred_idx] = 1
    output.backward(gradient=one_hot, retain_graph=True)

    # Calculate the importance weights
    importance_weights = torch.mean(model._modules[layer_name].weight.grad, axis=[2,3])

    # Calculate the heatmap
    heatmap = torch.zeros((h, w)).to(device)
    for i, weight in enumerate(importance_weights[0]):
        heatmap += weight * activations[layer_name][0, i, :, :]

    # Normalize the heatmap
    heatmap = np.maximum(heatmap.detach().cpu(), 0)
    heatmap /= torch.max(heatmap)

    return heatmap.numpy()


In [ ]:
# def find_target_layer(model):
#     """Find the final convolutional layer in the model"""
#     for name, module in model.named_modules():
#         if isinstance(module, nn.Conv2d):
#             if "classifier" in name:
#                 return module
#     raise ValueError("Could not find the target layer")


In [ ]:
# def find_target_layer(model):
#     # attempt to find the final convolutional layer in the network
#     # by looping over the layers of the network in reverse order
#     for layer in reversed(model.layers):
#         # check to see if the layer has a 4D output
#         if len(layer.output_shape) == 4:
#             return layer.name
#     # otherwise, we could not find a 4D layer so the GradCAM
#     # algorithm cannot be applied
#     raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")


In [ ]:
def find_target_layer(model):
    """Find the target layer in the model"""
    target_layer = None
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and module.out_features == 4:
            target_layer = module
            break
        elif isinstance(module, nn.Conv2d):
            target_layer = module
    if target_layer is None:
        raise ValueError("Could not find the target layer")
    return target_layer


In [ ]:
# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

test_iterator = iter(test_loader)
test_data = [next(test_iterator) for i in range(7)]

for i, (images, labels) in enumerate(test_data):
    images = images.to(device)

    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    for j, image in enumerate(images):
        # Generate the GradCAM heatmap
        target_layer = find_target_layer(efficientnet)
        heatmap = gradcam(image.unsqueeze(0), efficientnet, target_layer)
        heatmap = cv2.resize(heatmap, (224, 224))
        heatmap = heatmap - np.min(heatmap)
        heatmap = heatmap / np.max(heatmap)

        # Apply the heatmap to the original image
        image = image.cpu().numpy().transpose(1, 2, 0)
        image = cv2.resize(image, (224, 224))
        heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255
        image_with_heatmap = heatmap + np.float32(image)
        image_with_heatmap = image_with_heatmap / np.max(image_with_heatmap)

        # Get the predicted and true classes
                # Get the predicted and true classes
        with torch.no_grad():
            outputs = efficientnet(torch.unsqueeze(image, 0))
            _, preds = torch.max(outputs, 1)
        pred_class = class_names[preds[0].item()]
        true_class = class_names[labels[j]]

        # Plot the image with heatmap and label
        axs[j // 4, j % 4].imshow(image_with_heatmap)
        axs[j // 4, j % 4].set_title(f"Predicted: {pred_class}\nTrue: {true_class}")
        axs[j // 4, j % 4].axis("off")

    plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from gradcam import GradCAM, GradCAMpp


# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# Create an instance of the EfficientNet-B0 model
# efficientnet = models.efficientnet_b0(pretrained=True)
# efficientnet.to(device)

# Set the model to evaluation mode
# efficientnet.eval()

target_layer = efficientnet._fc

# Create an instance of the GradCAM class
gradcam = GradCAM( arch=efficientnet,target_layer=target_layer)

test_iterator = iter(test_loader)
test_data = [next(test_iterator) for i in range(7)]

print(dir(gradcam))
# print(type(gradcam))
print(images[j:j+1].shape)



for i, (images, labels) in enumerate(test_data):
    images = images.to(device)

    with torch.no_grad():
        outputs = efficientnet(images)
        _, preds = torch.max(outputs, 1)

    images = images.cpu().numpy()
    labels = labels.cpu().numpy()
    preds = preds.cpu().numpy()

    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    for j, image in enumerate(images):
        pred_class = class_names[preds[j]]
        true_class = class_names[labels[j]]
        axs[j // 4, j % 4].imshow(np.transpose(image, (1, 2, 0)))
        axs[j // 4, j % 4].set_title(f"Predicted: {pred_class}\nTrue: {true_class}")
        axs[j // 4, j % 4].axis("off")

        # Generate heatmap using forward method
        output = gradcam.forward(images[j:j+1])
#       output = gradcam(images[j:j+1])
        pred_class_idx = preds[j]
        pred_class_output = output[:, pred_class_idx]
        gradcam.backward(pred_class_output)
        heatmap = gradcam.generate()

        # Visualize heatmap on the original image
        superimposed_image = np.transpose(image, (1, 2, 0))
        superimposed_image = np.uint8(255 * superimposed_image)
        heatmap = cv2.resize(heatmap, (superimposed_image.shape[1], superimposed_image.shape[0]))
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        superimposed_image = cv2.addWeighted(superimposed_image, 0.5, heatmap, 0.5, 0)
        axs[j // 4, j % 4 + 4].imshow(superimposed_image)
        axs[j // 4, j % 4 + 4].axis("off")

    plt.show()

# Remove hooks to prevent memory leaks
gradcam.remove_hooks()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from gradcam import GradCAMpp

# Define a dictionary that maps label indices to class names
class_names = {0: "heavy plastic", 1: "no image", 2: "no_plastic", 3: "some_plastic"}

# Set the model to evaluation mode
efficientnet.eval()

# Create an instance of the GradCAMpp class
gradcam_pp = GradCAMpp(model=efficientnet, target_layer='blocks[-1].project.conv.weight')

test_iterator = iter(test_loader)
test_data = [next(test_iterator) for i in range(7)]

for i, (images, labels) in enumerate(test_data):
    images = images.to(device)

    with torch.no_grad():
        outputs = efficientnet(images)
        _, preds = torch.max(outputs, 1)

    images = images.cpu().numpy()
    labels = labels.cpu().numpy()
    preds = preds.cpu().numpy()

    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    for j, image in enumerate(images):
        pred_class = class_names[preds[j]]
        true_class = class_names[labels[j]]
        axs[j // 4, j % 4].imshow(np.transpose(image, (1, 2, 0)))
        axs[j // 4, j % 4].set_title(f"Predicted: {pred_class}\nTrue: {true_class}")
        axs[j // 4, j % 4].axis("off")

        # Generate GradCAMpp heatmap
        heatmap = gradcam_pp.generate_heatmap(images[j:j+1], target_class=preds[j])

        # Visualize heatmap on the original image
        superimposed_image = np.transpose(image, (1, 2, 0))
        superimposed_image = np.uint8(255 * superimposed_image)
        heatmap = cv2.resize(heatmap, (superimposed_image.shape[1], superimposed_image.shape[0]))
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        superimposed_image = cv2.addWeighted(superimposed_image, 0.5, heatmap, 0.5, 0)
        axs[j // 4, j % 4 + 4].imshow(superimposed_image)
        axs[j // 4, j % 4 + 4].axis("off")

    plt.show()

# Remove hooks to prevent memory leaks
gradcam_pp.remove_hooks()
